# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 10:32AM,247143,20,ALU0007987,Alumier Labs Inc.,Released
1,Sep 23 2022 10:32AM,247143,20,ALU0007994,Alumier Labs Inc.,Released
2,Sep 23 2022 10:32AM,247143,20,ALU0007995,Alumier Labs Inc.,Released
3,Sep 23 2022 10:32AM,247143,20,ALUR176077994,Alumier Labs Inc.,Released
4,Sep 23 2022 10:32AM,247143,20,ALUR474981228,Alumier Labs Inc.,Released
5,Sep 23 2022 10:32AM,247143,20,ALUR833234443,Alumier Labs Inc.,Released
6,Sep 23 2022 10:32AM,247141,16,EMERSA-8563966,"Emersa Waterbox, LLC",Released
7,Sep 23 2022 10:14AM,247142,19,60017724,"GUSA Granules USA, Inc.",Released
8,Sep 23 2022 10:14AM,247059,21,628362,"NBTY Global, Inc.",Executing
9,Sep 23 2022 10:10AM,247140,10,PRF-37770,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,247139,Released,9
31,247140,Released,6
32,247141,Released,1
33,247142,Released,1
34,247143,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247139,NaN,NaN,9.0
247140,NaN,NaN,6.0
247141,NaN,NaN,1.0
247142,NaN,NaN,1.0
247143,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0.0,1.0,0.0
246999,0.0,1.0,0.0
247000,0.0,1.0,0.0
247048,2.0,1.0,32.0
247059,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0,1,0
246999,0,1,0
247000,0,1,0
247048,2,1,32
247059,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,0,1,0
1,246999,0,1,0
2,247000,0,1,0
3,247048,2,1,32
4,247059,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,,1,
1,246999,,1,
2,247000,,1,
3,247048,2,1,32
4,247059,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.
6,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC"
7,Sep 23 2022 10:14AM,247142,19,"GUSA Granules USA, Inc."
8,Sep 23 2022 10:14AM,247059,21,"NBTY Global, Inc."
9,Sep 23 2022 10:10AM,247140,10,Bio-PRF
15,Sep 23 2022 10:09AM,247139,10,Eywa Pharma Inc.
24,Sep 23 2022 9:53AM,247116,21,"NBTY Global, Inc."
25,Sep 23 2022 9:50AM,247138,10,Cosmo International Fragrances
26,Sep 23 2022 9:43AM,247137,10,ISDIN Corporation
36,Sep 23 2022 9:29AM,247134,19,"Graystone, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.,,,6
1,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC",,,1
2,Sep 23 2022 10:14AM,247142,19,"GUSA Granules USA, Inc.",,,1
3,Sep 23 2022 10:14AM,247059,21,"NBTY Global, Inc.",,1,
4,Sep 23 2022 10:10AM,247140,10,Bio-PRF,,,6
5,Sep 23 2022 10:09AM,247139,10,Eywa Pharma Inc.,,,9
6,Sep 23 2022 9:53AM,247116,21,"NBTY Global, Inc.",,,1
7,Sep 23 2022 9:50AM,247138,10,Cosmo International Fragrances,,,1
8,Sep 23 2022 9:43AM,247137,10,ISDIN Corporation,,1,9
9,Sep 23 2022 9:29AM,247134,19,"Graystone, LLC",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.,6,,
1,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC",1,,
2,Sep 23 2022 10:14AM,247142,19,"GUSA Granules USA, Inc.",1,,
3,Sep 23 2022 10:14AM,247059,21,"NBTY Global, Inc.",,1,
4,Sep 23 2022 10:10AM,247140,10,Bio-PRF,6,,
5,Sep 23 2022 10:09AM,247139,10,Eywa Pharma Inc.,9,,
6,Sep 23 2022 9:53AM,247116,21,"NBTY Global, Inc.",1,,
7,Sep 23 2022 9:50AM,247138,10,Cosmo International Fragrances,1,,
8,Sep 23 2022 9:43AM,247137,10,ISDIN Corporation,9,1,
9,Sep 23 2022 9:29AM,247134,19,"Graystone, LLC",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.,6,,
1,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC",1,,
2,Sep 23 2022 10:14AM,247142,19,"GUSA Granules USA, Inc.",1,,
3,Sep 23 2022 10:14AM,247059,21,"NBTY Global, Inc.",,1,
4,Sep 23 2022 10:10AM,247140,10,Bio-PRF,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.,6.0,NaN,NaN
1,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,Sep 23 2022 10:14AM,247142,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Sep 23 2022 10:14AM,247059,21,"NBTY Global, Inc.",NaN,1.0,NaN
4,Sep 23 2022 10:10AM,247140,10,Bio-PRF,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 10:32AM,247143,20,Alumier Labs Inc.,6.0,0.0,0.0
1,Sep 23 2022 10:32AM,247141,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,Sep 23 2022 10:14AM,247142,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Sep 23 2022 10:14AM,247059,21,"NBTY Global, Inc.",0.0,1.0,0.0
4,Sep 23 2022 10:10AM,247140,10,Bio-PRF,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2965476,95.0,15.0,5.0
16,494248,2.0,0.0,0.0
19,988468,6.0,0.0,0.0
20,741311,49.0,26.0,11.0
21,1482180,2.0,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2965476,95.0,15.0,5.0
1,16,494248,2.0,0.0,0.0
2,19,988468,6.0,0.0,0.0
3,20,741311,49.0,26.0,11.0
4,21,1482180,2.0,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,95.0,15.0,5.0
1,16,2.0,0.0,0.0
2,19,6.0,0.0,0.0
3,20,49.0,26.0,11.0
4,21,2.0,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,95.0
1,16,Released,2.0
2,19,Released,6.0
3,20,Released,49.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Completed,5.0,0.0,0.0,11.0,0.0
Executing,15.0,0.0,0.0,26.0,4.0
Released,95.0,2.0,6.0,49.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Completed,5.0,0.0,0.0,11.0,0.0
1,Executing,15.0,0.0,0.0,26.0,4.0
2,Released,95.0,2.0,6.0,49.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Completed,5.0,0.0,0.0,11.0,0.0
1,Executing,15.0,0.0,0.0,26.0,4.0
2,Released,95.0,2.0,6.0,49.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()